Quantifying uncertainty is an important aspect of argumentation in the computational humanities when the measurements we take involve finite samples of data. This notebook explores two forms of confidence intervals for classification accuracy: binomial CIs and CIs calculated using the bootstrap.  We'll use them to generate confidence intervals for the accuracies in Underwood (2018), "The Life Cycle of Genres" using data available to support that article: [Underwood Github repo](https://github.com/tedunderwood/fiction).  

In [1]:
from sklearn import linear_model
import pandas as pd
from scipy import sparse
import numpy as np
from math import sqrt 
from scipy.stats import norm
from random import choices

import random
import csv
from os import path

from sklearn.model_selection import GroupKFold

random.seed(0)

In [2]:
def read_meta(filename, folder):

    detective_tags=set(['det100'])
    negative_tags = set(['random', 'chirandom'])
    
    df = pd.read_csv(filename)
    
    pos_ids={}
    neg_ids={}
    
    for index, row in df.iterrows():
        doc_id=row["docid"]
        author=row["author"]
        genres=set(x.strip() for x in row["genretags"].split("|"))
        
        if len(detective_tags.intersection(genres)) > 0 and path.exists("%s/%s.fic.tsv" % (folder, doc_id)):
            pos_ids[doc_id]=author

        if len(negative_tags.intersection(genres)) > 0 and len(detective_tags.intersection(genres)) == 0  and path.exists("%s/%s.fic.tsv" % (folder, doc_id)):
            neg_ids[doc_id]=author

    # Choose equal number of random documents for negative labels
    negs=list(neg_ids.keys())
    random.shuffle(negs)
    neg_ids_final={}
    for doc_id in negs[:len(pos_ids)]:
        neg_ids_final[doc_id]=neg_ids[doc_id]
    return pos_ids, neg_ids_final

In [3]:
pos_ids, neg_ids=read_meta("../data/underwood_genre_data/finalmeta.csv", "../data/underwood_genre_data/newdata")
print(len(pos_ids), len(neg_ids))

89 89


In [4]:
def read_vocab(filename):
    vocab={}
    df = pd.read_csv(filename)
    for index, row in df.iterrows():
            vocab[row[0]]=len(vocab)
    return vocab

In [5]:
vocab=read_vocab("../data/underwood_genre_data/new10k.csv")

In [6]:
def features_to_ids(data, feature_vocab):

    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [7]:
def read_data(folder, pos_ids, neg_ids, vocab):
    X=[]
    Y=[]
    M=[]
    
    for doc_id in pos_ids:
        filename="%s/%s.fic.tsv" % (folder, doc_id)
        feats={}
        with open(filename, encoding="utf-8") as file:
            for line in file:
                cols=line.rstrip().split("\t")
                word=cols[0]
                # word count is here if we need it but we'll use a binary flag instead
                count=int(cols[1])
                feats[word]=1

        X.append(feats)
        Y.append("detective")
        M.append(pos_ids[doc_id])

    for doc_id in neg_ids:
        filename="%s/%s.fic.tsv" % (folder, doc_id)
        feats={}
        with open(filename, encoding="utf-8") as file:
            for line in file:
                cols=line.rstrip().split("\t")
                word=cols[0]
                # word count is here if we need it but we'll use a binary flag instead
                count=int(cols[1])
                feats[word]=1
        
        X.append(feats)
        Y.append("random")
        M.append(neg_ids[doc_id])
    
    alldata=list(zip(X, Y, M))
    random.shuffle(alldata)
    shufX, shufY, shufM = zip(*alldata)
    
    trainX_ids=features_to_ids(shufX, vocab)

    return trainX_ids, np.array(shufY), np.array(shufM)

In [10]:
X, Y, M = read_data("../data/underwood_genre_data/newdata", pos_ids, neg_ids, vocab)

In [11]:
# We'll use GroupKFold to ensure that different texts by the same author don't appear in both the train 
# and test partition since it would be too easy (most works by the same author fall in the same genre)

kf = GroupKFold(n_splits=10)

predictions=[]
truth=[]

for train_index, test_index in kf.split(X, Y, M):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    clf = linear_model.LogisticRegression(C=1, solver='lbfgs', penalty='l2', max_iter=10000)
    clf.fit(X_train, y_train)
    
    print("Accuracy: %.3f" % clf.score(X_test, y_test))
    predictions.extend(clf.predict(X_test))
    truth.extend(y_test)

Accuracy: 0.778
Accuracy: 1.000
Accuracy: 0.944
Accuracy: 0.944
Accuracy: 0.889
Accuracy: 0.778
Accuracy: 1.000
Accuracy: 0.944
Accuracy: 0.941
Accuracy: 0.882


First, let's calculate parametric confidence intervals; these are appropriate where the central limit theorem holds (e.g., for metrics that involve averaging, like accuracy).

In [12]:
def binomial_confidence_intervals(predictions, truth, confidence_level=0.95):
    correct=[]
    for pred, gold in zip(predictions, truth):
        correct.append(int(pred==gold))
        
    success_rate=np.mean(correct)

    # two-tailed test
    critical_value=(1-confidence_level)/2
    # ppf finds z such that p(X < z) = critical_value
    z_alpha=-1*norm.ppf(critical_value)
    
    # the standard error is the square root of the variance/sample size
    # the variance for a binomial test is p*(1-p)
    standard_error=sqrt((success_rate*(1-success_rate))/len(correct))

    lower=success_rate-z_alpha*standard_error
    upper=success_rate+z_alpha*standard_error
    print("%.3f, %s%% Confidence interval: [%.3f,%.3f]" % (success_rate, confidence_level*100, lower, upper))

In [13]:
def accuracy(truth, predictions):
    correct=0.
    for idx in range(len(truth)):
        g=truth[idx]
        p=predictions[idx]
        if g == p:
            correct+=1
    return correct/len(truth)

In [14]:
binomial_confidence_intervals(predictions, truth, confidence_level=0.95)

0.910, 95.0% Confidence interval: [0.868,0.952]


Next, let's calculate confidence intervals using the bootstrap, which doesn't require assumptions on the parametric form.  This means it can be used for just about any metric (including those that don't involve averaging -- like the F-score).  How do these bounds compare to the parametric ones for accuracy?

In [15]:
def bootstrap(gold, predictions, metric, B=10000, confidence_level=0.95):
    critical_value=(1-confidence_level)/2
    lower_sig=100*critical_value
    upper_sig=100*(1-critical_value)
    data=[]
    for g, p in zip(gold, predictions):
        data.append([g,p])

    accuracies=[]
    
    for b in range(B):
        choice=choices(data, k=len(data))
        choice=np.array(choice)
        accuracy=metric(choice[:,0], choice[:,1])
        
        accuracies.append(accuracy)
    
    percentiles=np.percentile(accuracies, [lower_sig, 50, upper_sig])
    
    lower=percentiles[0]
    median=percentiles[1]
    upper=percentiles[2]
    
    return lower, median, upper


In [16]:
confidence_level=0.95
lower, median,upper=bootstrap(truth, predictions, accuracy, B=10000,confidence_level=confidence_level)
print("%.3f, %s%% Bootstrap confidence interval: [%.3f, %.3f]" % (median, confidence_level*100, lower, upper))

0.910, 95.0% Bootstrap confidence interval: [0.865, 0.949]
